# Connect to Weights and Biases Servers

In [1]:
!pip install wandb -qqq
import wandb
wandb.login() # Log in to your W&B account

wandb: Currently logged in as: ntulshibagwale (ae-ml_ucsb). Use `wandb login --relogin` to force relogin


True

# Import Modules, Set Random Seeds, and Access the Power of Google Colab

In [3]:
import os
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from torch.utils.data import Dataset
from torch import tensor
import torch.nn as nn
from tqdm.notebook import tqdm
import re, json, requests

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'tqdm'

# Define Machine Learning Pipeline and Utility Functions

In [2]:
def model_pipeline(config=None): 

    # tell wandb to get started
    with wandb.init(project=None, config=None):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      print("HYPER-Parameters:")
      for key,value in config.items():
        print(key, ':', value)
      print("")

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config)
      print(model)
      
      print('-----------------------------------------------------------------')
      print("Begin model training...\n")
      # and use them to train the model
      train(model, train_loader, criterion, optimizer, config)
      print("Training completed.\n")
      print('-----------------------------------------------------------------')
      
      print("Begin model evaluation...\n")
      print("\nEvaluate results on test data...\n")
      # and test its final performance
      test(model, test_loader, config)


    return model

def make(config):
    # Make the data
    train, test = get_data(config) 
    y_test = torch.argmax(test[:][1], dim=1)
    y_train = torch.argmax(train[:][1], dim=1)
    wandb.sklearn.plot_class_proportions(y_train,y_test, config.angles)
  
    train_loader = make_loader(train, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)
  
    # Make the model
    model = NeuralNetwork_01(config.feature_dim, config.classes).to(device)

    # Make the loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)
    
    # Look at shapes of batches
    for ex_idx, (batch_x,batch_y) in enumerate(train_loader):
        print(f"Shape of x batch is: {batch_x.shape}")
        print(f"Datatype of x batch is: {batch_x.dtype}\n")
        print(f"Shape of y batch is: {batch_y.shape}")
        print(f"Datatype of y batch is: {batch_y.dtype}\n")
        break

    return model, train_loader, test_loader, criterion, optimizer

def get_data(config):

    ae_dataset = AcousticEmissionDataset(config.url,
                                         config.sig_len,
                                         config.dt,
                                         config.low_pass,
                                         config.high_pass,
                                         config.fft_units,
                                         config.num_bins)
    
    total_count = len(ae_dataset)
    test_percent = 1 - config.train_percent
    train_count = int(config.train_percent * total_count)
    test_count = total_count - train_count
    torch.manual_seed(config.train_seed) 
    train, test = torch.utils.data.random_split(ae_dataset,
                                                      (train_count,
                                                        test_count))

    return train, test

def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

def train(model, loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, etc
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (waves, targets) in enumerate(loader):

            loss = train_batch(waves, targets, model, optimizer, criterion)
            example_ct +=  len(waves)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss, example_ct, epoch)

def train_batch(waves, targets, model, optimizer, criterion):
    waves, targets = waves.to(device), targets.to(device)

    # Forward pass ➡
    outputs = model(waves)
    loss = criterion(outputs, targets)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

def flatten(t):
    return [item for sublist in t for item in sublist]

def test(model, test_loader, config):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
      test_predicted = []
      test_targets = []
      correct, total = 0, 0
      i = 0
      for waves, targets in test_loader:
        waves, targets = waves.to(device), targets.to(device)
        outputs = model(waves)
        _, predicted = torch.max(outputs, 1)
        _, targets = torch.max(targets, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        test_predicted.append(predicted) # ex. [0,0,1]
        test_targets.append(targets)

        print(f"Accuracy of the model on the {total} " +
              f"test waves: {100 * correct / total}%")
        
    print("")
    print("Test Set Metrics:")
    # Obnoxious coding to get into some correct format
    test_predicted = [t.numpy() for t in test_predicted]
    test_targets = [t.numpy() for t in test_targets]
    #test_predicted = np.array(test_predicted).flatten()
    test_targets = flatten(test_targets)
    test_predicted = flatten(test_predicted)
    print(classification_report(test_predicted,test_targets,
                                target_names=config.angles))
    
    metrics = classification_report(test_predicted,test_targets,
                                target_names=config.angles,output_dict=True)
    wandb.log(metrics)  
    wandb.log({"test_accuracy": correct / total})
    
    torch.save(model.state_dict(), 'model.pth')
    wandb.save("model.pth")

def load_PLB(url):

    url = 'https://raw.githubusercontent.com/ntulshibagwale/plb_dataset_supervised_learning/main/Data/220426_PLB_data.json'
    resp = requests.get(url)
    PLB = json.loads(resp.text)
    for key in PLB.keys():
        PLB[key]  = np.array(PLB[key])
    return PLB


# Define AE Pencil Lead Break Dataset

In [4]:
class AcousticEmissionDataset(Dataset):
    
    # Constructor
    def __init__(self,url,sig_len,dt,low_pass,high_pass,fft_units,
                 num_bins):
      
        self.url = url
        self.sig_len = sig_len
        self.dt = dt
        self.low_pass = low_pass
        self.high_pass = high_pass
        self.fft_units = fft_units
        self.num_bins = num_bins
        
        # Load in AE Data
        data = load_PLB(url)
        
        # Separate dict into arrays
        waves = data['data']           # List of raw waveforms
        targets = data['target']       # waveform labels
        self.angles = data['target_angle']  # angle, one hot encoded
        self.targets = tensor(targets,dtype=torch.int64,requires_grad=False)
      
        # One hot encode
        targets_one_hot = tensor(targets,dtype=torch.int64,requires_grad=False)
        targets_one_hot = torch.nn.functional.one_hot(targets_one_hot.long()) 
        self.targets_one_hot = targets_one_hot.float()
        print("targets_one_hot is the one hot encoding for angle. ex: [1 0 1]")

        self.n_samples = self.targets.shape[0]    # Number of samples/labels
        self.waves = tensor(waves,dtype=torch.float32,requires_grad=False)                          
                   
        print("")
        print(f"Shape of waves is: {self.waves.shape}")
        print(f"Datatype of waves is: {self.waves.dtype}")
        print("waves requires grad:", self.waves.requires_grad)
        print(f"Shape of targets is: {self.targets.shape}")
        print(f"Datatype of targets is: {self.targets.dtype}")
        print("targets requires grad:", self.targets.requires_grad)
        print(f"Ex: {targets[0]}")
        print(f"Shape of targets_one_hot is: {self.targets_one_hot.shape}")
        print(f"Datatype of targets_one_hot is: {self.targets_one_hot.dtype}")
        print("targets_one_hot requires grad:", \
              self.targets_one_hot.requires_grad)
        print(f"Ex: {targets_one_hot[0]}")
        print("")
        
        print("AcousticEmissionDataset loaded in!\n")
        
    def __getitem__(self,index):
        """
        
        Function called when object is indexed. The transformation of data 
        occurs in this getter function. In other words, the constructor reads
        in the raw data filtered by hand, and this function contains the 
        sequence of remaining processing on the data to extract features used
        in ML models.
        
        index (int): the index of the sample and label
        
        return:
        (x,y): feature vector and label corresponding to single event
        
        """
        x = self.waves[index]      
        y = self.targets_one_hot[index] 
              
        return x, y # input example, label
    
    def __len__(self): # number of samples
        return self.n_samples
  
    def _get_angle_subset(self, specific_angles):
        """
        
        Get subset of AE data based on angle.

        Parameters
        ----------
        specific_angles : array-like
            List of angles that a sub-dataset will be composed of.

        Returns
        -------
        subset : Pytorch Dataset Object
            Object for training / testing, containing specified angles.

        """
        targets = self.targets # use int label for which angle it is
        indices = []
        for idx, target in enumerate(targets): # loop through all data
            if self.angles[target] in specific_angles:
                indices.append(idx) # if angle is a match remember index
        
        subset = torch.utils.data.Subset(self,indices) # get subset
        
        return subset

# Define Network Architecture

In [5]:
class NeuralNetwork_01(nn.Module): 
    # classification
    def __init__(self, feature_dim, classes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(feature_dim, 20),
            nn.ReLU(),
            nn.Linear(20,classes)
        )

    def forward(self, x):
        z = self.layers(x)
        return z # predictions

## Enter Hyperparameters and Train Model

In [6]:
sweep_config = {
  "name" : "sweep2",
  "method" : "bayes",
  "metric" : {
    'goal': 'maximize',
    'name': 'accuracy'
  },
  "parameters" : {
    "epochs" : {
      "min": 1000,
      "max": 2000
    },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.01
    },
    "classes" : {
        'value' : 6
    },
    "batch_size" : {
        "values" : [10,50,128]
    },
    "train_percent" : {
        "value" : 0.8
    },
    "url" : {
        "value" : 'https://raw.githubusercontent.com/ntulshibagwale/plb_dataset_supervised_learning/main/Data/220426_PLB_data.json'
    },
    'architecture' : {
        "value" : 1
    },
    "feature_dim" : {
        "value" : 1024
    },
    "sig_len" : {
        "value" : 1024
    },
    "dt" : {
        "value" : 10**-7
    },
    "low_pass" : {
        "value" : 0
    },
    "high_pass" : {
        "value" : 100*10**4
    },
    "fft_units" : {
        "value" : 1000
    },
    "num_bins" : {
        "value" : 26
    },
    "train_seed" : {
        "value" : 41
    },
    "angles" : {
        "value" : ['20deg', '22deg', '26deg', '30deg', '36deg', '40deg']
    }
  }
}

sweep_id = wandb.sweep(sweep_config,project="plb-dataset")
wandb.agent(sweep_id, function=model_pipeline, count=10)

Create sweep with ID: gjo2tho0
Sweep URL: https://wandb.ai/ae-ml_ucsb/plb-dataset/sweeps/gjo2tho0


wandb: Agent Starting Run: y2tk2nzq with config:
wandb: 	angles: ['20deg', '22deg', '26deg', '30deg', '36deg', '40deg']
wandb: 	architecture: 1
wandb: 	batch_size: 128
wandb: 	classes: 6
wandb: 	dt: 1e-07
wandb: 	epochs: 1097
wandb: 	feature_dim: 1024
wandb: 	fft_units: 1000
wandb: 	high_pass: 1000000
wandb: 	learning_rate: 0.00874416963166661
wandb: 	low_pass: 0
wandb: 	num_bins: 26
wandb: 	sig_len: 1024
wandb: 	train_percent: 0.8
wandb: 	train_seed: 41
wandb: 	url: https://raw.githubusercontent.com/ntulshibagwale/plb_dataset_supervised_learning/main/Data/220426_PLB_data.json
wandb: Currently logged in as: ntulshibagwale (ae-ml_ucsb). Use `wandb login --relogin` to force relogin


HYPER-Parameters:
angles : ['20deg', '22deg', '26deg', '30deg', '36deg', '40deg']
architecture : 1
batch_size : 128
classes : 6
dt : 1e-07
epochs : 1097
feature_dim : 1024
fft_units : 1000
high_pass : 1000000
learning_rate : 0.00874416963166661
low_pass : 0
num_bins : 26
sig_len : 1024
train_percent : 0.8
train_seed : 41
url : https://raw.githubusercontent.com/ntulshibagwale/plb_dataset_supervised_learning/main/Data/220426_PLB_data.json

targets_one_hot is the one hot encoding for angle. ex: [1 0 1]

Shape of waves is: torch.Size([514, 1024])
Datatype of waves is: torch.float32
waves requires grad: False
Shape of targets is: torch.Size([514])
Datatype of targets is: torch.int64
targets requires grad: False
Ex: 0
Shape of targets_one_hot is: torch.Size([514, 6])
Datatype of targets_one_hot is: torch.float32
targets_one_hot requires grad: False
Ex: tensor([1, 0, 0, 0, 0, 0])

AcousticEmissionDataset loaded in!

Shape of x batch is: torch.Size([128, 1024])
Datatype of x batch is: torch.fl

  0%|          | 0/1097 [00:00<?, ?it/s]

Loss after 02466 examples: 0.081
Loss after 05060 examples: 0.090
Loss after 07654 examples: 0.082
Loss after 10248 examples: 0.082
Loss after 12741 examples: 0.080
Loss after 15335 examples: 0.074
Loss after 17929 examples: 0.071
Loss after 20523 examples: 0.069
Loss after 23016 examples: 0.079
Loss after 25610 examples: 0.061
Loss after 28204 examples: 0.056
Loss after 30798 examples: 0.051
Loss after 33291 examples: 0.073
Loss after 35885 examples: 0.044
Loss after 38479 examples: 0.042
Loss after 41073 examples: 0.051
Loss after 43566 examples: 0.047
Loss after 46160 examples: 0.046
Loss after 48754 examples: 0.034
Loss after 51348 examples: 0.044
Loss after 53841 examples: 0.028
Loss after 56435 examples: 0.031
Loss after 59029 examples: 0.033
Loss after 61623 examples: 0.028
Loss after 64116 examples: 0.029
Loss after 66710 examples: 0.036
Loss after 69304 examples: 0.024
Loss after 71898 examples: 0.040
Loss after 74391 examples: 0.031
Loss after 76985 examples: 0.029
Loss after

...

Loss after 01122 examples: 0.120
Loss after 02255 examples: 0.099
Loss after 03388 examples: 0.093
Loss after 04521 examples: 0.087
Loss after 05693 examples: 0.092
Loss after 06826 examples: 0.083
Loss after 07959 examples: 0.087
Loss after 09092 examples: 0.082
Loss after 10264 examples: 0.077
Loss after 11397 examples: 0.079
Loss after 12530 examples: 0.085
Loss after 13663 examples: 0.085
Loss after 14796 examples: 0.080
Loss after 15968 examples: 0.073
Loss after 17101 examples: 0.079
Loss after 18234 examples: 0.078
Loss after 19367 examples: 0.067
Loss after 20539 examples: 0.068
Loss after 21672 examples: 0.064
Loss after 22805 examples: 0.065
Loss after 23938 examples: 0.058
Loss after 25071 examples: 0.073
Loss after 26243 examples: 0.057
Loss after 27376 examples: 0.072
Loss after 28509 examples: 0.057
Loss after 29642 examples: 0.067
Loss after 30814 examples: 0.061
Loss after 31947 examples: 0.058
Loss after 33080 examples: 0.055
Loss after 34213 examples: 0.059
Loss after

wandb: Agent Starting Run: iz39lnbv with config:
wandb: 	angles: ['20deg', '22deg', '26deg', '30deg', '36deg', '40deg']
wandb: 	architecture: 1
wandb: 	batch_size: 10
wandb: 	classes: 6
wandb: 	dt: 1e-07
wandb: 	epochs: 1024
wandb: 	feature_dim: 1024
wandb: 	fft_units: 1000
wandb: 	high_pass: 1000000
wandb: 	learning_rate: 0.0006709223526074385
wandb: 	low_pass: 0
wandb: 	num_bins: 26
wandb: 	sig_len: 1024
wandb: 	train_percent: 0.8
wandb: 	train_seed: 41
wandb: 	url: https://raw.githubusercontent.com/ntulshibagwale/plb_dataset_supervised_learning/main/Data/220426_PLB_data.json


HYPER-Parameters:
angles : ['20deg', '22deg', '26deg', '30deg', '36deg', '40deg']
architecture : 1
batch_size : 10
classes : 6
dt : 1e-07
epochs : 1024
feature_dim : 1024
fft_units : 1000
high_pass : 1000000
learning_rate : 0.0006709223526074385
low_pass : 0
num_bins : 26
sig_len : 1024
train_percent : 0.8
train_seed : 41
url : https://raw.githubusercontent.com/ntulshibagwale/plb_dataset_supervised_learning/main/Data/220426_PLB_data.json

targets_one_hot is the one hot encoding for angle. ex: [1 0 1]

Shape of waves is: torch.Size([514, 1024])
Datatype of waves is: torch.float32
waves requires grad: False
Shape of targets is: torch.Size([514])
Datatype of targets is: torch.int64
targets requires grad: False
Ex: 0
Shape of targets_one_hot is: torch.Size([514, 6])
Datatype of targets_one_hot is: torch.float32
targets_one_hot requires grad: False
Ex: tensor([1, 0, 0, 0, 0, 0])

AcousticEmissionDataset loaded in!

Shape of x batch is: torch.Size([10, 1024])
Datatype of x batch is: torch.fl

  0%|          | 0/1024 [00:00<?, ?it/s]

Loss after 00240 examples: 0.122
Loss after 00481 examples: 0.112
Loss after 00731 examples: 0.114
Loss after 00972 examples: 0.089
Loss after 01222 examples: 0.118
Loss after 01463 examples: 0.106
Loss after 01704 examples: 0.107
Loss after 01954 examples: 0.099
Loss after 02195 examples: 0.092
Loss after 02445 examples: 0.080
Loss after 02686 examples: 0.116
Loss after 02927 examples: 0.070
Loss after 03177 examples: 0.100
Loss after 03418 examples: 0.102
Loss after 03668 examples: 0.082
Loss after 03909 examples: 0.079
Loss after 04150 examples: 0.103
Loss after 04400 examples: 0.102
Loss after 04641 examples: 0.077
Loss after 04891 examples: 0.071
Loss after 05132 examples: 0.075
Loss after 05373 examples: 0.098
Loss after 05623 examples: 0.088
Loss after 05864 examples: 0.074
Loss after 06114 examples: 0.096
Loss after 06355 examples: 0.085
Loss after 06596 examples: 0.080
Loss after 06846 examples: 0.093
Loss after 07087 examples: 0.067
Loss after 07337 examples: 0.100
Loss after

In [70]:
# Inputs
config = dict(
    epochs=1000,
    classes=6,
    batch_size=128,
    learning_rate=0.005,
    url = 'https://raw.githubusercontent.com/ntulshibagwale/plb_dataset_supervised_learning/main/Data/220426_PLB_data.json',
    architecture= 1,
    feature_dim = 1024, # raw waveform
    sig_len = 1024,
    dt = 10**-7,
    low_pass = 0,
    high_pass = 100*10**4,
    fft_units = 1000,
    num_bins=26,
    train_seed = 41, # THIS ENSURES SAME SPLIT FOR TEST DATA EVERY RUN
    train_percent = 0.8,
    angles = ['20deg', '22deg', '26deg', '30deg', '36deg', '40deg']
    )

# Build, train and analyze the model with the pipeline
model = model_pipeline(config)

TypeError: ignored